In [12]:
import pandas as pd
import requests 
import json 
import csv 
import time
import datetime
import time

In [2]:
import time
import datetime
s = "04/12/2020"
str(int(time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())))


'1607029200'

In [14]:
"""Search Settings here"""

after = '06/03/2020'
before = '10/05/2022'

query = "GME|gamestop" #Keyword(s) to look for in submissions
sub = "wallstreetbets" #Which Subreddit to search in

after = str(int(time.mktime(datetime.datetime.strptime(after, "%d/%m/%Y").timetuple()))) #Submissions after this timestamp 
before = str(int(time.mktime(datetime.datetime.strptime(before, "%d/%m/%Y").timetuple()))) #Submissions before this timestamp



In [16]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [19]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    if 'selftext' in subm.keys():
        selftext = subm['selftext']
    else:
        selftext = " "
    permalink = subm['permalink']

    subData.append((sub_id,title,url,author,score,created,numComms,selftext,permalink,flair))
    subStats[sub_id] = subData

In [ ]:

subCount = 0
subStats = {}

data = getPushshiftData(query, after, before, sub)


while len(data) > 1: 
    for submission in data:
        collectSubData(submission)
        subCount+=1
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    print(after)
    while True:
        try:
            data = getPushshiftData(query, after, before, sub) 
            break
        except ValueError:
            print("error")
            continue  # ignore

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))


In [7]:
def updateSubs_file(attempt):
    upload_count = 0
    filename =str('posts_') + attempt+'.csv' 
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","selftext","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")



In [8]:
updateSubs_file('first')

180578 submissions have been uploaded
